In [98]:
"""Getting all relevent imports into the file"""
import re
import pandas as pd #imported pandas to use dataframes
import numpy as np #imported numpy for dynamic arrays
import matplotlib.pyplot as plt # imported pyplot to use with my visuals
import requests #imported to use webscrape
from requests import get #pulled get to use for scraping
from random import randint #pulled randint to use as a buffer for time
from time import sleep #pulled sleep to slow the crawl rate of the scraper
from bs4 import BeautifulSoup #pulled to webscrape

In [157]:
"""Pulling information from the actual IMDb website."""
pages = np.arange(1,40556,250)
title = []
movie_rating = []
year =  []
star_rating = []
gross = []
movie_genre = []
movie_time = []
votes = []
def scraper():
    for page in pages:
        page = requests.get('https://www.imdb.com/search/title/?release_date=1950-01-01,2020-12-31&certificates=US%3AG,US%3APG,US%3APG-13,US%3AR,US%3ANC-17&sort=alpha,asc&count=250&start='+ str(page) +'&ref_=adv_nxt')
        
        soup = BeautifulSoup(page.text, 'html.parser')
        
#pulling from lister-item mode-advanced pulls all the information for each of the movies on the page to easily pull 
#the Html tags as attributes of an item
        container = soup.findAll('div',class_ = 'lister-item mode-advanced') 
        """ for loop looking for each of the catagories in each of the containers"""
        for name in container:
            """nv is so we can seperate the gross income and seperate it from the votes set into their own """
            nv = name.find_all('span',attrs ={'name':'nv'})
            
            title.append(name.h3.a.text)
            movie_rating.append(name.p.span.text)
            year.append(name.h3.find('span',class_ = 'lister-item-year').text)
            if name.strong is not None:
                star_rating.append(float(name.strong.text))
            else:
                star_rating.append(0)
            gross_income = nv[1].text.strip('$\n\tM') if len(nv) > 1 else '0'
            gross.append(gross_income)
            votes_total = nv[0].text if len(nv) >1 else '0'
            votes.append(votes_total)
            if name.p.find('span',class_ = 'genre') is not None:
                movie_genres = name.p.find('span',class_ ='genre').text.strip(',\n\t" "').split(',')
                movie_genre.append(list(map(lambda ele: ele.split(','),movie_genres)))#movie_genres
            else:
                movie_genre.append('N/A')
            if name.p.find('span',class_ = 'runtime') is None:
                movie_time.append(0)
            else:
                movie_time.append(name.p.find('span', class_ = 'runtime').text)

In [208]:
"""Cleaning of the information. removing unneeded characters out of the set of information as well as setting up the dataframe"""
def Clean():
    df.loc[:,'Year'] = df['Year'].astype(str).str.strip('()TV Special Movie VideoGame TV–\I Short" "X')#.str.lstrip('\(\)')#.astype(int)
    df.loc[:,'Movie Runtime'] = df['Movie Runtime'].astype(str).str.rstrip('min').astype(int)
    df.loc[:,'Votes'] = df['Votes'].astype(str).str.replace(',','').astype(float)
    df.loc[:,'Gross Income']= df['Gross Income'].astype(str).str.replace('-'," ",True).astype(float)
    df.loc[:, 'Title'] = df['Title'].astype(str)
    df.loc[:,'Movie Genre'] = df['Movie Genre'].astype(str)
    df.drop_duplicates()
    

In [159]:
"""Pull and clean information to get ready for setting to dataframe"""
scraper()


In [210]:
"""Setting the information from above to a dataframe to manipulate and visualize"""
df = pd.DataFrame({'Title': title,'Movie Rating': movie_rating,'Year': year,
                   'Gross Income': gross,'Movie Genre': movie_genre, 'Movie Runtime': movie_time,
                    'IMDb Rating': star_rating,'Votes': votes})
Clean()
df.head()
#

,Title,Movie Rating,Year,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
0,"""Flip Flop Flippin': One man's search for char...",PG,2014,0.0,[['Documentary']],85,5.9,0.0
1,Pop Wrapped TV,G,2009,0.0,"[['Documentary'], [' News']]",0,0.0,0.0
2,Laura,PG-13,2015,0.0,[['Comedy']],21,0.0,0.0
3,#Illusion,PG,2014,0.0,"[['Short'], [' Drama']]",13,6.6,0.0
4,#TheLateBatsby,G,2018,0.0,"[['Animation'], [' Short'], [' Comedy']]",4,5.4,0.0


In [171]:
df['Movie Genre'].unique()

array(["[['Documentary']]", "[['Documentary'], [' News']]",
       "[['Comedy']]", "[['Short'], [' Drama']]",
       "[['Animation'], [' Short'], [' Comedy']]",
       "[['Comedy'], [' Crime'], [' Drama']]",
       "[['Comedy'], [' Family'], [' Fantasy']]", "[['Reality-TV']]",
       "[['Comedy'], [' Drama']]",
       "[['Action'], [' Adventure'], [' Comedy']]",
       "[['Animation'], [' Drama'], [' Fantasy']]",
       "[['Action'], [' Crime'], [' Thriller']]", "[['Drama']]",
       "[['Action'], [' Crime'], [' Drama']]",
       "[['Action'], [' Comedy'], [' Crime']]",
       "[['Drama'], [' History'], [' War']]", "[['Western']]",
       "[['Action'], [' Adventure'], [' Drama']]",
       "[['Crime'], [' Drama'], [' Thriller']]",
       "[['Drama'], [' Music']]", "[['Romance'], [' Comedy']]",
       "[['Short'], [' Family']]", "[['Short'], [' Music']]",
       "[['Short'], [' Comedy'], [' Music']]",
       "[['Comedy'], [' Drama'], [' Romance']]",
       "[['Comedy'], [' Drama'], [' Mu

In [70]:
df.set_index('Title')

,Movie Rating,Year,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
Title,,,,,,,
500 Days of Summer,PG-13,2009,32.39,"['Comedy', ' Drama', ' Romance']",95,7.7,461968.0
12 Years a Slave,R,2013,56.67,"['Biography', ' Drama', ' History']",134,8.1,620057.0
127 Hours,R,2010,18.34,"['Biography', ' Drama']",94,7.6,340533.0
1917,R,2019,159.23,"['Drama', ' War']",119,8.3,341593.0
2001: A Space Odyssey,G,1968,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583079.0
...,...,...,...,...,...,...,...
Cold War,R,2018,4.57,"['Drama', ' History', ' Music']",89,7.6,44907.0
Zodiac,R,2007,33.08,"['Crime', ' Drama', ' Mystery']",157,7.7,437514.0
Zombieland,R,2009,75.59,"['Adventure', ' Comedy', ' Fantasy']",88,7.6,507625.0


In [211]:
df.to_csv('All_movies_year_1950-2020.csv')